In [ ]:
import requests
import spacy

In [ ]:
api_url = "http://127.0.0.1:8000"

In [ ]:
# model_name = "en_ner_craft_md"
model_name = "en_ner_bc5cdr_md"

In [ ]:
ner = spacy.load(model_name)

In [ ]:
ner.pipe_names

In [ ]:
doc_ner = ner("Alterations in the hypocretin receptor 2 and preprohypocretin genes produce narcolepsy in some animals.")

for ent in doc_ner.ents:
    print(ent.text, ent.label_, ent.start_char, ent.end_char, ent.start, ent.end)

In [ ]:
doc_ner.to_json()

In [ ]:
ner.meta

In [ ]:
r = requests.get("http://127.0.0.1:8000/documents/?skip=0&limit=100")
j = r.json()
j

In [ ]:
ner_keys = ["title", "summary"]
source_name = "scispacy_" + model_name

In [ ]:
for doc in j:
    # print(doc["id"])
    for key in ner_keys:
        # print(key)
        doc_ner = ner(doc[key])

        for ent in doc_ner.ents:
            # print("\t", ent.text, ent.label_, ent.start_char, ent.end_char, ent.start, ent.end)
            entity_id = ent.label_ + ":" + ent.text
            entity_id = entity_id.replace("/", "_").strip().replace(" ", "_").lower()
            
            # check if entity exists and register if non-existant
            r = requests.get(f"{api_url}/entities/{entity_id}")
            if r.status_code == 404:
                entity_post_json = {
                  "id": entity_id,
                  "preferred_name": ent.text,
                  "entity_type": ent.label_,
                  "synonyms": [
                    ent.text
                  ],
                  "source": source_name
                }
                r = requests.post(f"{api_url}/entities/", json=entity_post_json)
                assert r.status_code == 200
            
            # check if mention exists and register if non-existant
            r = requests.get(f"{api_url}/entities/{entity_id}/documents/{doc['id']}")
            j = r.json()
            starts_ends = {(x['start_char'], x['end_char'], x['source']) for x in j}
            if (ent.start_char, ent.end_char, source_name) not in starts_ends:
                mention_json = {
                  "document_id": doc["id"],
                  "entity_id": entity_id,
                  "text": ent.text,
                  "document_section": key,
                  "start_char": ent.start_char,
                  "end_char": ent.end_char,
                  "start_token": ent.start,
                  "end_token": ent.end,
                  "source": source_name,
                }
                r = requests.post(f"{api_url}/entity_mentions/", json=mention_json)
                assert r.status_code == 200